# Import libraries and model hyperparameters

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import argparse
import random
random.seed(1)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dropout, Flatten, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Define model hyperparameters
parser = argparse.ArgumentParser(description='CNN-LSTM Upper Bound')
parser.add_argument('--input_size', type=int, default=7)
parser.add_argument('--batch_size', type=int, default=128)
parser.add_argument('--num_epochs', type=int, default=100)
parser.add_argument('--learning_rate', type=float, default=0.001)
parser.add_argument('--input_features', type=list, default=['Mode',
                                                            'Indoor Temp',
                                                            'Indoor Humidity',
                                                            'Air Velocity',
                                                            'Globe Temperature',
                                                            'Outdoor Temp',
                                                            'Outdoor Humidity'])
args = parser.parse_args(args=[])

mode_mapping = {'AC':0, 'NV':1}
thermalpref_mapping = {'No Change':0, 'Warmer':1, 'Cooler':2}
thermalacc_mapping = {'Acceptable':0, 'Unacceptable':1}
airpref_mapping = {'No Change':0, 'More':1, 'Less':2}

# Load data

In [ ]:
# load training data from BCA dataset
thermalpref_train = pd.read_csv('../data/bca_thermalpref_train_data.csv')
thermalacc_train = pd.read_csv('../data/bca_thermalacc_train_data.csv')
airpref_train = pd.read_csv('../data/bca_airpref_train_data.csv')

# load test data from BCA dataset
thermalpref_test = pd.read_csv('../data/bca_thermalpref_test_data.csv')
thermalacc_test = pd.read_csv('../data/bca_thermalacc_test_data.csv')
airpref_test = pd.read_csv('../data/bca_airpref_test_data.csv')

In [ ]:
# perform one hot encoding
thermalpref_train['Mode'] = thermalpref_train['Mode'].apply(lambda x: mode_mapping[x])
thermalpref_test['Mode'] = thermalpref_test['Mode'].apply(lambda x: mode_mapping[x])

thermalacc_train['Mode'] = thermalacc_train['Mode'].apply(lambda x: mode_mapping[x])
thermalacc_test['Mode'] = thermalacc_test['Mode'].apply(lambda x: mode_mapping[x])

airpref_train['Mode'] = airpref_train['Mode'].apply(lambda x: mode_mapping[x])
airpref_test['Mode'] = airpref_test['Mode'].apply(lambda x: mode_mapping[x])

In [ ]:
# perform label mapping
thermalpref_train['Thermal Preference'] = thermalpref_train['Thermal Preference'].apply(lambda x: thermalpref_mapping[x])
thermalpref_test['Thermal Preference'] = thermalpref_test['Thermal Preference'].apply(lambda x: thermalpref_mapping[x])

thermalacc_train['Thermal Acceptability'] = thermalacc_train['Thermal Acceptability'].apply(lambda x: thermalacc_mapping[x])
thermalacc_test['Thermal Acceptability'] = thermalacc_test['Thermal Acceptability'].apply(lambda x: thermalacc_mapping[x])

airpref_train['Air Movement Preference'] = airpref_train['Air Movement Preference'].apply(lambda x: airpref_mapping[x])
airpref_test['Air Movement Preference'] = airpref_test['Air Movement Preference'].apply(lambda x: airpref_mapping[x])

# Define model architecture

In [ ]:
def define_model_architecture(num_classes):
    """
    Defines the CNN LSTM model architecture with model parameters.
    
    Parameters:
        num_classes: An integer value indicating the number of output classes for the model.
        
    Return:
        model: The Keras object containing the CNN LSTM model architecture.
    """

    # Create a Sequential model
    model = Sequential()

    # 1D Convolutional Layer (Part of the model to be retrained)
    model.add(Conv1D(filters=128, kernel_size=5, padding='same', input_shape=(args.input_size, 1), trainable=True))
    model.add(Dropout(0.1))

    # Two LSTM Layers (Part of the model to be retrained)
    model.add(LSTM(256, return_sequences=True, recurrent_dropout=0.1, trainable=True))
    model.add(LSTM(256, return_sequences=False, recurrent_dropout=0.1, trainable=True))

    # Flatten the output from LSTM layers
    model.add(Flatten())

    # Two Dense (Fully Connected) Layers (These layers will remain fixed)
    model.add(Dense(64, activation='relu', kernel_initializer='glorot_uniform', trainable=True))
    model.add(Dense(16, activation='relu', kernel_initializer='glorot_uniform', trainable=True))

    # Output Layer (Part of the model to be retrained)
    model.add(Dense(num_classes, activation='softmax', trainable=True))

    # Compile the model
    optimizer = tf.keras.optimizers.Adam(learning_rate=args.learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    # Print model summary
    print(model.summary())
    
    return model

# Model training and evaluation

In [ ]:
def train_model(model, train_df, model_name, target_col, num_classes):
    # Assuming train_df has columns for features and a 'target' column for labels
    X = np.array(train_df[args.input_features])
    y = np.array(train_df[target_col])

    # Split the data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
    
    # Create and fit a Min-Max scaler
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    # Convert labels to one-hot encoding
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
    y_val = tf.keras.utils.to_categorical(y_val, num_classes=num_classes)

    # Define a callback to save the best model during training
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint('model.h5', save_best_only=True)
    
    # Early stopping callback
    early_stopping_callback = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',  # Monitor validation loss
        patience=5,          # Number of epochs with no improvement after which training will be stopped
        restore_best_weights=True  # Restore the best model weights when training stops
    )

    # Train the model
    history = model.fit(X_train_scaled, 
                        y_train, 
                        epochs=args.num_epochs, 
                        batch_size=args.batch_size,
                        validation_data=(X_val_scaled, y_val), 
                        callbacks=[checkpoint_callback])

    # Save the final trained model
    model.save(model_name + '.h5')
    
    print(history)

    return model, scaler

def evaluate_model(model, test_df, scaler, model_name, target_col, metrics=['accuracy', 'weighted_f1']):
    # Assuming test_df has columns for features and a 'target' column for labels
    X_test = np.array(test_df[args.input_features])
    y_true = np.array(test_df[target_col])

    # Load the pre-trained model
    loaded_model = tf.keras.models.load_model(model_name + '.h5')
    
    # Apply numerical scaler on X_test
    X_test_scaled = scaler.transform(X_test)

    # Get predictions from the model
    y_pred = loaded_model.predict(X_test_scaled)
    y_pred_classes = np.argmax(y_pred, axis=1)

    evaluation_results = {}

    if 'accuracy' in metrics:
        accuracy = accuracy_score(y_true, y_pred_classes)
        evaluation_results['accuracy'] = accuracy

    if 'weighted_f1' in metrics:
        weighted_f1 = f1_score(y_true, y_pred_classes, average='weighted')
        evaluation_results['weighted_f1'] = weighted_f1

    return evaluation_results


In [ ]:
# Train and evaluate thermal preference model (source: N.A., target: BCA)
print("Training Thermal Preference Model")
thermalpref_model = define_model_architecture(num_classes=3)
thermalpref_model, thermalpref_scaler = train_model(thermalpref_model, 
                                                    thermalpref_train, 
                                                    model_name='cnnlstm_upper_thermalpref_model', 
                                                    target_col='Thermal Preference', 
                                                    num_classes=3)

thermalpref_eval = evaluate_model(thermalpref_model, 
                                  thermalpref_test, 
                                  thermalpref_scaler,
                                  model_name='cnnlstm_upper_thermalpref_model', 
                                  target_col='Thermal Preference')
print("Thermal Preference Accuracy:", thermalpref_eval['accuracy'])
print("Thermal Preference Weighted F1 Score:", thermalpref_eval['weighted_f1'])

In [ ]:
# Train and evaluate thermal acceptability model (source: N.A., target: BCA)
print("Training Thermal Acceptability Model")
thermalacc_model = define_model_architecture(num_classes=2)
thermalacc_model, thermalacc_scaler = train_model(thermalacc_model, 
                                                  thermalacc_train, 
                                                  model_name='cnnlstm_upper_thermalacc_model', 
                                                  target_col='Thermal Acceptability', 
                                                  num_classes=2)

thermalacc_eval = evaluate_model(thermalacc_model, 
                                 thermalacc_test, 
                                 thermalacc_scaler,
                                 model_name='cnnlstm_upper_thermalacc_model', 
                                 target_col='Thermal Acceptability')
print("Thermal Acceptability Accuracy:", thermalacc_eval['accuracy'])
print("Thermal Acceptability Weighted F1 Score:", thermalacc_eval['weighted_f1'])

In [ ]:
# Train and evaluate air movement preference model (source: N.A., target: BCA)
print("Training Air Movement Preference Model")
airpref_model = define_model_architecture(num_classes=3)
airpref_model, airpref_scaler = train_model(airpref_model, 
                                            airpref_train, 
                                            model_name='cnnlstm_upper_airpref_model', 
                                            target_col='Air Movement Preference', 
                                            num_classes=3)

airpref_eval = evaluate_model(airpref_model, 
                              airpref_test, 
                              airpref_scaler,
                              model_name='cnnlstm_upper_airpref_model', 
                              target_col='Air Movement Preference')
print("Air Movement Preference Accuracy:", airpref_eval['accuracy'])
print("Air Movement Preference Weighted F1 Score:", airpref_eval['weighted_f1'])